In [28]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder


store_data = pd.read_csv('/Users/peytonbailey/WGU Data Science Masters/D599- Data Preparation and Exploration/Task 3/Megastore_Dataset_Task_3 3.csv')

In [29]:
store_data.head()

,OrderID,ProductName,Quantity,InvoiceDate,UnitPrice,TotalCost,Country,DiscountApplied,OrderPriority,Region,Segment,ExpeditedShipping,PaymentMethod,CustomerOrderSatisfaction
0,536370,INFLATABLE POLITICAL GLOBE,48,12/1/2010 8:45,$0.85,$40.80,United States,Yes,High,Northeast,Corporate,Yes,Credit Card,Satisfied
1,536370,SET2 RED RETROSPOT TEA TOWELS,18,12/1/2010 8:45,$2.95,$53.10,United States,Yes,High,Northeast,Corporate,Yes,Credit Card,Satisfied
2,536370,PANDA AND BUNNIES STICKER SHEET,12,12/1/2010 8:45,$0.85,$10.20,United States,Yes,High,Northeast,Corporate,Yes,Credit Card,Satisfied
3,536370,RED TOADSTOOL LED NIGHT LIGHT,24,12/1/2010 8:45,$1.65,$39.60,United States,Yes,High,Northeast,Corporate,Yes,Credit Card,Satisfied
4,536370,VINTAGE HEADS AND TAILS CARD GAME,24,12/1/2010 8:45,$1.25,$30.00,United States,Yes,High,Northeast,Corporate,Yes,Credit Card,Satisfied


In [30]:
#Transactionalize data
transactions = store_data.groupby('OrderID')['ProductName'].apply(list).reset_index()

In [31]:
print(transactions)



     OrderID                                        ProductName
0     536370  [INFLATABLE POLITICAL GLOBE , SET2 RED RETROSP...
1     536852  [POLKADOT RAIN HAT , VINTAGE HEADS AND TAILS C...
2     536974  [EDWARDIAN PARASOL RED, LUNCH BAG RED RETROSPO...
3     537065  [PARTY TIME PENCIL ERASERS, RED RETROSPOT PURS...
4     537463  [PINK POLKADOT CHILDRENS UMBRELLA, RED RETROSP...
..       ...                                                ...
436   581001  [RETROSPOT CHILDRENS APRON, STRAWBERRY LUNCH B...
437   581171  [RED RETROSPOT CHILDRENS UMBRELLA, RED RETROSP...
438   581279  [CAKE STAND WHITE TWO TIER LACE, REGENCY CAKES...
439   581316  [RED RETROSPOT SUGAR JAM BOWL, GLASS  SONGBIRD...
440   581587  [BAKING SET 9 PIECE RETROSPOT , CHILDRENS APRO...

[441 rows x 2 columns]


In [32]:
#TransactionEncoder object
encoder = TransactionEncoder()

# Fit and transform the transactions
onehot = encoder.fit(transactions['ProductName']).transform(transactions['ProductName'])

# Convert the array to a DataFrame
onehot = pd.DataFrame(onehot, columns=encoder.columns_)

onehot = onehot.astype(int)

#Print
print(onehot)

onehot.to_excel('/Users/peytonbailey/WGU Data Science Masters/D599- Data Preparation and Exploration/Task 3/transactions.xlsx',index=False)

      50S CHRISTMAS GIFT BAG LARGE   DOLLY GIRL BEAKER  \
0                                0                   0   
1                                0                   0   
2                                0                   0   
3                                0                   0   
4                                0                   0   
..                             ...                 ...   
436                              0                   0   
437                              0                   0   
438                              0                   0   
439                              0                   0   
440                              0                   0   

      I LOVE LONDON MINI BACKPACK   NINE DRAWER OFFICE TIDY  \
0                               0                         0   
1                               0                         0   
2                               0                         0   
3                               0                  

In [33]:
#Apriori
freq_items=apriori(onehot,min_support=0.05,use_colnames=True)


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [34]:
#Generate association rules
rules = association_rules(freq_items, metric="lift", min_threshold=1)
rules= rules.sort_values(by=["lift", "confidence", "support"], ascending=False)
print(rules[['antecedents', 'consequents', 'support', 'lift', 'confidence']].head(10))



                                          antecedents  \
6                       (CHILDRENS CUTLERY SPACEBOY )   
7                     (CHILDRENS CUTLERY DOLLY GIRL )   
37  (ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...   
40                        (ALARM CLOCK BAKELIKE RED )   
38  (ALARM CLOCK BAKELIKE RED , ALARM CLOCK BAKELI...   
39                       (ALARM CLOCK BAKELIKE GREEN)   
3                         (ALARM CLOCK BAKELIKE RED )   
2                        (ALARM CLOCK BAKELIKE GREEN)   
36  (ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...   
41                        (ALARM CLOCK BAKELIKE PINK)   

                                          consequents   support       lift  \
6                     (CHILDRENS CUTLERY DOLLY GIRL )  0.056689  14.080460   
7                       (CHILDRENS CUTLERY SPACEBOY )  0.056689  14.080460   
37                        (ALARM CLOCK BAKELIKE RED )  0.056689  10.274930   
40  (ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...  0.056

In [35]:
#Narrow down top 3 rules
top_rules = rules[['antecedents', 'consequents', 'support', 'lift', 'confidence']].head(3)
print(top_rules)



                                          antecedents  \
6                       (CHILDRENS CUTLERY SPACEBOY )   
7                     (CHILDRENS CUTLERY DOLLY GIRL )   
37  (ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...   

                        consequents   support      lift  confidence  
6   (CHILDRENS CUTLERY DOLLY GIRL )  0.056689  14.08046    0.925926  
7     (CHILDRENS CUTLERY SPACEBOY )  0.056689  14.08046    0.862069  
37      (ALARM CLOCK BAKELIKE RED )  0.056689  10.27493    0.862069  
